### Dependencies

In [1]:
from owlready2 import *
#Shoudl be imported after owlready
from rdflib import Graph

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


### Methods to get the entity labels

In [2]:
def getClasses(onto):        
    return onto.classes()
    
def getDataProperties(onto):        
    return onto.data_properties()
    
def getObjectProperties(onto):        
    return onto.object_properties()
    
def getIndividuals(onto):    
    return onto.individuals()


def getRDFSLabelsForEntity(entity):
    #if hasattr(entity, "label"):
    return entity.label


def getRDFSLabelsForEntity(entity):
    #if hasattr(entity, "label"):
    return entity.label 

### Load ontology and print labels

lexical similarity, rdflib to create triples, predicates

In [3]:
#Load ontology and print 5 classes with labels

#Conference ontologies do not contain rdfs:label but a URI with a readable name
urionto="data/cmt.owl"
#urionto="data/ekaw.owl"
#urionto="data/confOf.owl"

#human and mouse contain rdfs:label but the URI has a concept ID
#urionto="data/human.owl"
#urionto="data/mouse.owl


#Method from owlready
onto = get_ontology(urionto).load()
    
print("Classes in Ontology: " + str(len(list(getClasses(onto)))))
i=0
for cls in getClasses(onto):
    i+=1
    #Name of entity in URI. But in some cases it may be a 
    #code like in mouse and human anatomy ontologies                
    print(cls.iri)
    print("\t"+cls.name)  
    #Labels from RDFS label
    print("\t"+str(getRDFSLabelsForEntity(cls)))
    
    if i==5:
        break

Classes in Ontology: 29
http://cmt#Meta-Reviewer
	Meta-Reviewer
	[]
http://cmt#Reviewer
	Reviewer
	[]
http://cmt#Decision
	Decision
	[]
http://cmt#Person
	Person
	[]
http://cmt#Document
	Document
	[]


In [6]:
#Load ontology and print 5 classes with labels

#Conference ontologies do not contain rdfs:label but a URI with a readable name
#urionto="data/cmt.owl"
urionto="data/ekaw.owl"
#urionto="data/confOf.owl"

#human and mouse contain rdfs:label but the URI has a concept ID
#urionto="data/human.owl"
#urionto="data/mouse.owl


#Method from owlready
onto2 = get_ontology(urionto).load()
    
print("Classes in Ontology: " + str(len(list(getClasses(onto2)))))
i=0
for cls in getClasses(onto2):
    i+=1
    #Name of entity in URI. But in some cases it may be a 
    #code like in mouse and human anatomy ontologies                
    print(cls.iri)
    print("\t"+cls.name)  
    #Labels from RDFS label
    print("\t"+str(getRDFSLabelsForEntity(cls)))
    
    if i==5:
        break

Classes in Ontology: 73
http://ekaw#Event
	Event
	[]
http://ekaw#Person
	Person
	[]
http://ekaw#Accepted_Paper
	Accepted_Paper
	[]
http://ekaw#Rejected_Paper
	Rejected_Paper
	[]
http://ekaw#Evaluated_Paper
	Evaluated_Paper
	[]


    Get URI from ... from lab4? and use directly in g.add. Then use rdflib to export to turtle. 

In [19]:
from isub import isub
from rdflib import Namespace
from rdflib.namespace import OWL
a = Namespace()
g = Graph()
g.add()
def lexical_similarity(onto1,onto2):
    for cls1 in getClasses(onto1):
        for cls2 in getClasses(onto2):
            if isub(cls1.name,cls2.name) > 0.8:
                g.add(cls1.uri)
                print('cmt:'+cls1.name+' '+'owl:equivalentClass'+' ekaw:'+cls2.name)
lexical_similarity(onto,onto2)

cmt:Meta-Reviewer owl:equivalent ekaw:Review
cmt:Reviewer owl:equivalent ekaw:Review
cmt:Reviewer owl:equivalent ekaw:Possible_Reviewer
cmt:Person owl:equivalent ekaw:Person
cmt:Document owl:equivalent ekaw:Document
cmt:Preference owl:equivalent ekaw:Presenter
cmt:Conference owl:equivalent ekaw:Conference_Paper
cmt:Conference owl:equivalent ekaw:Conference_Session
cmt:Conference owl:equivalent ekaw:Conference_Trip
cmt:Conference owl:equivalent ekaw:Conference_Participant
cmt:Conference owl:equivalent ekaw:Conference
cmt:Conference owl:equivalent ekaw:Conference_Banquet
cmt:Conference owl:equivalent ekaw:Conference_Proceedings
cmt:ConferenceChair owl:equivalent ekaw:Conference_Paper
cmt:ConferenceChair owl:equivalent ekaw:Conference_Trip
cmt:ConferenceChair owl:equivalent ekaw:Conference
cmt:ConferenceMember owl:equivalent ekaw:Conference_Paper
cmt:ConferenceMember owl:equivalent ekaw:Conference_Trip
cmt:ConferenceMember owl:equivalent ekaw:Conference
cmt:Review owl:equivalent ekaw:Revi

In [23]:
from isub import isub
def lexical_similarity(onto1,onto2):
    for cls1 in getObjectProperties(onto1):
        for cls2 in getObjectProperties(onto2):
            if isub(cls1.name,cls2.name) > 0.8:
                print('cmt:'+cls1.name+' '+'owl:equivalentProperty'+' ekaw:'+cls2.name)
lexical_similarity(onto,onto2)

cmt:assignedByReviewer owl:equivalent ekaw:hasReviewer
cmt:assignReviewer owl:equivalent ekaw:hasReview
cmt:assignReviewer owl:equivalent ekaw:hasReviewer
cmt:reviewCriteriaEnteredBy owl:equivalent ekaw:reviewWrittenBy
cmt:writtenBy owl:equivalent ekaw:writtenBy
cmt:writtenBy owl:equivalent ekaw:reviewWrittenBy


In [24]:
from isub import isub
def lexical_similarity(onto1,onto2):
    for cls1 in getIndividuals(onto1):
        for cls2 in getIndividuals(onto2):
            if isub(cls1.name,cls2.name) > 0.8:
                print('cmt:'+cls1.name+' '+'owl:sameAs'+' ekaw:'+cls2.name)
lexical_similarity(onto,onto2)

### Method to compute Precision and Recall

In [4]:
def compareWithReference(reference_mappings_file, system_mappings_file):
    ref_mappings = Graph()
    ref_mappings.parse(reference_mappings_file, format="ttl")
    
    system_mappings = Graph()
    system_mappings.parse(system_mappings_file, format="ttl")
    
    
    #We calculate precision and recall via true positives, false positives and false negatives
    #https://en.wikipedia.org/wiki/Precision_and_recall        
    tp=0
    fp=0
    fn=0
    
    for t in system_mappings:
        if t in ref_mappings:
            tp+=1
        else:
            fp+=1

    
    for t in ref_mappings:
        if not t in system_mappings:
            fn+=1
            
            
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = (2*precision*recall)/(precision+recall)
    #print(tp, tp2)
    #print(fp)
    #print(fn)
    print("Comparing '" + system_mappings_file + "' with '" + reference_mappings_file)
    print("\tPrecision: " + str(precision))
    print("\tRecall: " + str(recall))
    print("\tF-Score: " + str(f_score))

### Check results

In [5]:
reference_mappings="data/anatomy-reference-mappings.ttl"
system_mappings="data/anatomy-example-system-mappings.ttl"

#P, R, and F can only be obtained if a reference alignment exists.    
compareWithReference(reference_mappings, system_mappings)

# For the lab you should compare, for example, 
# cmt-confOf-reference.ttl with the cmt-confOf-your-system.ttl you generate.
# compareWithReference("cmt-confOf-reference.ttl", "cmt-confOf-your-system.ttl")


Comparing 'data/anatomy-example-system-mappings.ttl' with 'data/anatomy-reference-mappings.ttl
	Precision: 0.8537434706906558
	Recall: 0.9703166226912929
	F-Score: 0.9083050324174129
